# Data Preperation
The resulting data from this notebook will combine the positional data (latitude, longitude, town) of the ChemDataForJeffOlson.csv and all of the tables from the Land use survey.
## Merge all tables from the land use survey
Here we run through the files from the land use survey. All files have the same colum names and thus we just need to append them all. I created a dataframe from the first file, then ran through the rest of them turning each into a dataframe then appending it to the first.

In [4]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from urllib.request import urlretrieve
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [19]:
# set the path to the folder with all the tables
land_use_folder = 'assets/Tables'

# set the path to the chem data file
chem_data_file_path = 'assets/chem_data.csv'

# set the path to the population data (Optional)
USE_POPULATION = True
population_path = 'assets/HS-STAT-Population-of-Vermont-towns-1930-2019.xls'

# set the path to the characteristics data
characteristics_path = 'assets/Characteristic.csv'

# set the save path to the resulting cleaned chem data
chem_data_save_path = 'assets/chem_data_merged.csv'

# set the save path to the resulting survey data file
survay_save_path = 'assets/combined_tables.csv'

# set geojson path
geojson_path = 'assets/Lakes_Inventory.geojson'

In [32]:
URL_HEALTH = 'https://www.healthvermont.gov/sites/default/files/documents/xls/HS-STAT-Population-of-Vermont-towns-1930-2019.xls'

URL_LAKES = 'https://anrmaps.vermont.gov/arcgis/rest/services/Open_Data/OPENDATA_ANR_WATER_SP_NOCACHE_v2/MapServer/208/query?where=1%3D1&outFields=*&outSR=4326&f=json'

# This fuction downloads the geojson data from VERMONT OPEN GEODATA PORTAL.
def get_lakes_inventory_geojson(filename=geojson_path, url=URL_LAKES):
    if not os.path.exists(filename):
        urlretrieve(url, filename)
    data = gpd.read_file(r'assets/Lakes_Inventory.geojson')
    return data

# This fuction downloads the population data.
def get_pop_data(filename=population_path, url=URL_HEALTH):
    if not os.path.exists(filename):
        urlretrieve(url, filename)
    
    
    population = pd.read_excel(population_path, skiprows=4, index_col='CTC')
    # lets take the relevent columns from the population dataset and melt them so we ca easily join the dataframes
    population = population[['NAME'] + list(population.columns[-32:])]
    population.rename(columns={'NAME': 'Town'}, inplace=True)
    return population

In [16]:
# Download geojson data for ploting in project dashboard notebook

geojson = get_lakes_inventory_geojson()

In [ ]:
# Some measurments have a very low frequency or have been measured only once or twice.
# Here we set a TRESHOLD parameter to cut off parameters with less than TRESHOLD measurments available in dataset.
 
# set treshold for number of measurments
TRESHOLD = 100

def get_frequent_measurments(df, TRESHOLD):
    counts = df.groupby(['CharacteristicID'])['Result'].count()
    mask = counts[counts.values <= TRESHOLD].index
    df = df.where(~df.isin(mask))
    return df

In [3]:
# combine all tables from Table Folder in assets folder

def merge_tables_folder(tables_folder):
    # get the first file as data frame to append to
    file_1 = os.listdir(tables_folder)[0]
    tables_df = pd.read_excel(os.path.join(tables_folder, file_1))
    
    # add file name to df
    tables_df['from_file'] = file_1
    
    # run through the rest of the files and append them to the data frame
    for file in os.listdir(tables_folder)[1:]:
        df = pd.read_excel(os.path.join(tables_folder, file))
        df['from_file'] = file
        tables_df = pd.concat((tables_df, df), axis=0)
    
    # drop OBJECTID column and reset index
    tables_df = tables_df.drop('OBJECTID', axis=1).reset_index(drop=True)
    return tables_df

In [4]:
combined_tables_df = merge_tables_folder(land_use_folder)

In [5]:
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
530,Buffer100ftWBFL_JOBS,4409.339846,6.491541e+04,14.535149,1.097826,0.000000,0.343600,0.062641,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.074248,0.000000,0.000000,0.000000,0.074248,0.000000,0.000000,12.178085,2.385524,14.563609,0.000000,7.904513,0.000000,7.904513,AOIs_JOBS.xls
630,SPECTACLE_Waterbody100ft,8847.418250,1.338072e+05,24.401742,4.411071,0.177607,2.539313,0.156850,0.117560,0.438858,0.814149,0.0,0.000000,0.000000,0.000000,0.274850,0.189747,0.551837,0.554199,0.843592,2.414224,0.000000,0.000000,10.213707,14.254045,24.467752,0.266036,4.202636,1.815811,6.284483,AOIs_SPECTACLE.xlsx
350,WOODWARD_Buffer100ftWBFL,24856.339856,5.997148e+05,129.109027,12.155960,0.122379,3.018388,0.244819,3.196489,0.331306,0.000000,0.0,0.656938,0.000000,0.656938,0.144936,0.276433,0.343567,3.752939,0.000000,4.517875,1.804519,1.804519,25.105377,104.088503,129.193880,0.042003,4.577352,1.762558,6.381914,AOIs_WOODWARD.xlsx
175,MILLWINDSR_Buffer100ftWBFL,395518.414644,1.221288e+07,2321.277559,546.729697,5.950227,85.805605,6.108559,36.111492,15.594549,0.000000,0.0,163.002574,131.391508,294.394083,6.728310,6.534510,17.290929,65.819552,0.000000,96.373300,23.010417,23.010417,740.330068,1584.864693,2325.194761,74.371604,352.235345,34.359390,460.966339,AOIs_MILLWINDSR.xlsx
702,AMHERST_Buffer250ftWaterbody,6806.416537,2.468990e+05,49.386776,6.098057,0.000000,1.673395,0.346441,2.799761,0.703817,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.431153,0.787496,4.425356,0.000000,5.644005,0.000000,0.000000,15.145244,34.346300,49.491544,1.574334,1.795248,1.503243,4.872826,AOIs_AMHERST.xls


## Split the Description column to get LakeIDs
The Description column has the LakeID attached to a description. depending on the lake they may be LakeID then description or description then LakeID. the function below splits on an underscore and takes the part of the newly created list that is all uppercase as the LakeID and leaves the rest as the description.

In [6]:
# now I want to solit the description column in to two columns 
# one corresponds to the title area and the other corresponds to the description
# depending on the folder they are in different orders

def split_description(df):
    df['Description'] = df['Description'].str.split('_')
    df['LakeID'] = df['Description'].apply(lambda row: row[0] if row[0].isupper() else row[1])
    df['Description'] = df['Description'].apply(lambda row: row[0] if not row[0].isupper() else row[1])
    return df

In [7]:
combined_tables_df = split_description(combined_tables_df)
combined_tables_df.sample(5)

,Description,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file,LakeID
18,Buffer100ftWBFL,66891.562577,1.979020e+06,288.649096,170.221121,0.266070,20.500078,1.928716,4.785868,2.615792,0.0,0.479587,89.928339,3.196648,93.604573,0.344409,2.126587,2.908508,5.908986,0.0,11.288490,23.844480,23.844480,167.928230,121.456491,289.384721,43.579724,119.240908,24.330610,187.151243,AOIs_PARKER.xls,PARKER
203,Flowline100ft,2799.634268,7.600817e+04,12.604579,5.495862,0.000000,0.680033,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.742805,0.742805,3.847913,8.798860,12.646774,5.259421,2.567994,0.703338,8.530753,AOIs_WAPANACKI.xlsx,WAPANACKI
698,Watershed,42913.416342,5.037842e+07,11697.119006,555.690342,14.257403,72.164668,15.218579,55.869205,38.430511,0.0,0.000000,27.522155,13.586335,41.108489,16.204539,16.733727,43.634701,99.099354,0.0,175.672321,129.284771,129.284771,1617.660606,10085.028263,11702.688869,115.520810,1053.222166,95.798530,1264.541506,AOIs_AMHERST.xls,AMHERST
751,Buffer100ftWBFL,105723.704562,3.073768e+06,394.678701,186.826268,1.467618,161.633172,4.210175,4.953405,5.700712,0.0,2.672151,67.774921,0.897419,71.344491,1.513427,4.579198,6.554423,8.552785,0.0,21.199834,26.851697,26.851697,144.506110,251.466239,395.972349,47.063689,99.621999,20.670011,167.355699,AOIs_LITTLEWELLS.xls,LITTLEWELLS
506,Buffer250ftWaterbody,34476.500174,1.281291e+06,240.342725,46.381856,0.322657,4.662624,10.047413,3.633247,11.194412,0.0,0.000000,3.114893,0.000000,3.114893,0.342851,12.613435,14.545890,10.434207,0.0,37.936384,0.000000,0.000000,92.615729,148.582486,241.198215,0.803006,8.836177,0.254092,9.893276,AOIs_DUNMORE.xls,DUNMORE


## Merge relevant columns from the chem data and population datasets to survey data
For now I will just take the 'LakeID', 'Lat', 'Long', 'Town' from the chem data dataset. for other use later i will also extract the measurements when I know what year the land use survey was done. I will need to fix the LakeIDs in the combined_tables_df dataframe so that they match up with the LakeIDs in the chem dataset. This is the column that i will join on.

In [ ]:
chem_data_df = pd.read_csv(chem_data_file_path)

In [ ]:
# set TRSHOLD and run this if you want to exclude infrequent measurments
TRESHOLD = 100
chem_data_df = get_frequent(chem_data_df, TRESHOLD)

In [8]:
chem_data_df.sample(5)

,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode
183476,OSMORE,1,Pelagic,44.30830,-72.27890,Peacham,AcidLake,1985-02-02 00:00:00,1,1215.0,PlasticKemm,1.0,Reg,TSO4,NaN,6.40,Y,NaN,NaN,NaN
224183,SOUTH (EDEN),1,Pelagic,44.68330,-72.52750,Eden,SpringTP,1999-04-26 00:00:00,1,1246.0,Hydrolab,1.0,R1,DO,NaN,10.34,Y,NaN,NaN,NaN
44779,CEDAR,1,Pelagic,44.25017,-73.13338,Monkton,SpringTP,2002-04-03 00:00:00,1,1040.0,Hydrolab,1.0,R1,TempC,NaN,7.44,Y,NaN,NaN,NaN
45094,CENTER,1,Pelagic,44.71534,-71.91882,Newark,SpringTP,2001-05-17 00:00:00,1,1311.0,Hydrolab,15.0,Reg,DO%,NaN,43.50,Y,NaN,NaN,NaN
49168,CLYDE,1,Pelagic,44.93560,-72.17220,Derby,SpringTP,2008-05-12 00:00:00,1,1601.0,Hydrolab,4.0,Reg,pH,NaN,7.39,Y,NaN,NaN,NaN


In [9]:
if USE_POPULATION:    
    population = get_pop_data()

In [10]:
# now lets convert town names in the chem dataset to uppercase to match the town names in the population data set
chem_data_df['Town'] = chem_data_df['Town'].str.upper()
# we also need to extract the year from the chem data set so that we can join the population data
# to the chem data by year and Town
chem_data_df['VisitDate'] = pd.to_datetime(chem_data_df['VisitDate'])
chem_data_df['year'] = chem_data_df['VisitDate'].dt.year

In [11]:
chem_to_add = chem_data_df[['LakeID', 'Lat', 'Long', 'Town']].copy()
chem_to_add.drop_duplicates(inplace=True)

# get all lake ids that are also in usage survey and fix them
for lake_id in np.unique(chem_to_add['LakeID']):
    lake_id_fixed = re.sub('[()\s;]', '', lake_id)
    index = combined_tables_df[combined_tables_df['LakeID'] == lake_id_fixed].index
    combined_tables_df.loc[index, 'LakeID'] = lake_id


In [12]:
# merge with the chem_to_add data frame
new_data_df = combined_tables_df.merge(chem_to_add, how='left', on='LakeID')

# reorder columns
cols = list(new_data_df.columns)

to_front = ['LakeID', 'Description', 'Lat', 'Long', 'Town']
for col in to_front:
    cols.remove(col)

cols = to_front + cols
new_data_df = new_data_df.reindex(columns=cols)

new_data_df.head(5)

,LakeID,Description,Lat,Long,Town,Shape_Length,Shape_Area,TREE_CANOPY_acres,GRASS_SHRUBS_acres,BARE_SOIL_acres,WATER_acres,BUILDINGS_acres,ROADS_acres,OTHER_PAVED_acres,RAILROADS_acres,Ag_Crops_acres,Ag_Hay_acres,Ag_Pasture_acres,Ag_Total_acres,Imp_Bare_Soil_acres,Imp_Buildings_acres,Imp_Other_Paved_acres,Imp_Road_acres,Imp_Railroad_acres,Imp_Total_acres,Shrub_Shrubs_acres,Shrub_Total_acres,TC_Coniferous_acres,TC_Deciduous_acres,TC_Total_acres,Wet_Emergent_acres,Wet_Forested_acres,Wet_Scrub_Shrub_acres,Wet_Total_acres,from_file
0,HORSE,Watershed,44.6169,-72.2108,GREENSBORO,9262.633620,2.107941e+06,468.319079,45.532062,0.034904,3.322265,0.043058,3.534652,0.097545,0.0,0.0,0.0,0.0,0.0,0.067911,0.042648,0.116058,5.195567,0.0,5.422184,0.946536,0.946536,210.794069,258.350785,469.144853,14.288349,77.086939,5.588561,96.963848,AOIs_HORSE.xls
1,HORSE,Flowline100ft,44.6169,-72.2108,GREENSBORO,1500.131136,4.176164e+04,4.545867,3.129955,0.000000,2.491621,0.000000,0.150858,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.151524,0.0,0.151524,0.279845,0.279845,2.970089,1.629099,4.599187,3.341299,2.996817,0.908847,7.246963,AOIs_HORSE.xls
2,HORSE,Waterbody100ft,44.6169,-72.2108,GREENSBORO,4447.434860,6.269552e+04,6.338120,8.104550,0.000000,0.834165,0.022425,0.093591,0.096186,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.114463,0.177650,0.0,0.314428,0.011841,0.011841,5.278762,1.092946,6.371707,5.845567,2.616113,2.488963,10.950643,AOIs_HORSE.xls
3,HORSE,Buffer100ftWBFL,44.6169,-72.2108,GREENSBORO,5715.509700,1.020007e+05,10.868234,10.706442,0.000000,3.262898,0.022425,0.244449,0.096186,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.114463,0.329174,0.0,0.465952,0.291685,0.291685,8.234746,2.720674,10.955420,8.581423,5.612931,3.397809,17.592163,AOIs_HORSE.xls
4,HORSE,Buffer250ftWaterbody,44.6169,-72.2108,GREENSBORO,4647.798280,1.545388e+05,23.735053,12.987901,0.000000,0.874999,0.022425,0.466102,0.097545,0.0,0.0,0.0,0.0,0.0,0.000000,0.022314,0.116058,0.614319,0.0,0.752692,0.604020,0.604020,16.834859,7.006856,23.841715,9.646655,8.976926,3.505087,22.128668,AOIs_HORSE.xls


In [13]:
# now since the land use survey was conducted from 2013 - 2016 I will merge the population data for 2016 joining on the town names
if use_population:    
    pop2016 = population[['Town', 2016]].copy()
    pop2016.rename(columns={2016: '2016_population'}, inplace=True)
    new_data_df = new_data_df.merge(pop2016, how='left', on='Town')
    print('Final survey data DataFrame shape: ', new_data_df.shape)
    new_data_df.sample(5)

Final survey data DataFrame shape:  (773, 36)


## Save the survey dataframe
Lets save the resulting dataframe as a .csv for later use

In [14]:
# save the data frame as a .csv
new_data_df.to_csv(survay_save_path)

## Merge the population dataset, the chem data dataset and the characteristics dataset
Here we will take the population dataset and melt it so that the years are also in the rows sp that we can merge on both town and year data.

In [15]:
# melt the population dataframe
if use_population:
    population_1 = population.melt(id_vars='Town', var_name='year', value_name='population')
    chem_data_df = chem_data_df.merge(population_1, how='left', on=['Town', 'year'])
    chem_data_df.sample(5)

In [16]:
# Now so that we have measurements
characteristics = pd.read_csv(characteristics_path)
characteristics.columns = ['CharacteristicID', 'CharacteristicName', 'UnitCode', 'SampleFraction']
characteristics.head()

,CharacteristicID,CharacteristicName,UnitCode,SampleFraction
0,AshFreeDryMass,Ash Free Dry Mass,mg,NaN
1,BOD5,"Biological Oxygen Demand, 5 day",mg/l,NaN
2,BottomDepth,Bottom depth,m,NaN
3,BottomSecchi,Bottom secchi,NaN,NaN
4,CBOD5,"Carbonaceous Biological Oxygen Demand, 5 day",mg/l,NaN


In [17]:
# we are going to merge on CharacteristicID so to make sure that they are the same across dataframes
# we will make all if them uppercase in both dataframes
characteristics['CharacteristicID'] = characteristics['CharacteristicID'].str.upper()
chem_data_df['CharacteristicID'] = chem_data_df['CharacteristicID'].str.upper()

In [18]:
# Merge characteristics df
chem_data_df_final = chem_data_df.merge(characteristics, how='left', on='CharacteristicID')
print('Final chem data Data frame shape: ', chem_data_df_final.shape)
chem_data_df_final.sample(5)

Final chem data Data frame shape:  (284912, 25)


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction
110382,HARVEYS,1,Pelagic,44.29390,-72.1375,BARNET,Laymon,1983-07-19,1,1015.0,Hose,8.0,Reg,CHLA,NaN,4.80,Y,NaN,NaN,NaN,1983,NaN,Chlorophyll-a,ug/l,NaN
256667,TICKLENAKED,1,Pelagic,44.19067,-72.0989,RYEGATE,LakeAsmt,2005-06-16,1,1040.0,Hydrolab,1.0,Reg,COND,NaN,174.40,Y,NaN,NaN,NaN,2005,1174.0,Conductivity,umho/cm,NaN
264840,TICKLENAKED,1,Pelagic,44.19067,-72.0989,RYEGATE,TMDL,2017-09-19,1,946.0,Hydrolab,3.0,Reg,COND,NaN,120.60,Y,NaN,NaN,NaN,2017,1107.0,Conductivity,umho/cm,NaN
249710,SUNSET (MARLBR),1,Pelagic,42.91780,-72.6833,MARLBORO,AcidLake,2007-07-26,1,750.0,PlasticKemm,10.0,Reg,DOC,NaN,1.64,Y,NaN,NaN,H,2007,1049.0,Dissolved Organic Carbon,mg/l,Dissolved
141854,LITTLE (WINHLL),1,Pelagic,43.12360,-72.9428,WINHALL,AcidLake,1986-06-05,1,900.0,BottleGrab,0.1,S1,PH,NaN,5.50,Y,NaN,NaN,NaN,1986,NaN,pH,None,NaN


In [19]:
# add in normalized results columns
for char_id in chem_data_df_final['CharacteristicID'].unique():
    df1 = chem_data_df_final[chem_data_df_final['CharacteristicID'] == char_id]
    chem_data_df_final.loc[df1.index, 'NormResult'] = MinMaxScaler().fit_transform(df1['Result'].values.reshape(-1, 1))
    
print(chem_data_df_final.shape)
chem_data_df_final.head()

(284912, 26)


,LakeID,LakeStationNo,LakeStationType,Lat,Long,Town,ProjectID,VisitDate,VisitNumber,StartTime,CollectionMethodID,Depth,ActivityCategory,CharacteristicID,Symbol,Result,Calcs,ProjRemark,RemarkCode,DepthStratumCode,year,population,CharacteristicName,UnitCode,SampleFraction,NormResult
0,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1988-04-22,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,2.8,Y,NaN,B,NaN,1988,2377.0,Secchi transparency,m,NaN,0.176694
1,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1988-04-22,1,NaN,Kemmerer,1.4,Reg,TP,NaN,9.0,Y,NaN,NaN,NaN,1988,2377.0,Total Phosphorus,ug/l,Total,0.004051
2,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1989-05-01,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,2.3,Y,NaN,B,NaN,1989,2417.0,Secchi transparency,m,NaN,0.145028
3,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1989-05-01,1,NaN,Kemmerer,1.0,Reg,TP,NaN,11.0,Y,NaN,NaN,NaN,1989,2417.0,Total Phosphorus,ug/l,Total,0.005208
4,ABENAKI,1,Pelagic,43.8303,-72.2361,THETFORD,SpringTP,1990-04-20,1,NaN,Secchi,NaN,Reg,SECCHI,NaN,3.1,Y,NaN,B,NaN,1990,2438.0,Secchi transparency,m,NaN,0.195693


## Save the chem data DataFrame
Lets save the resulting dataframe as a .csv for later use

In [20]:
chem_data_df_final.to_csv(chem_data_save_path)

## Documantation

In [21]:
%load_ext watermark
%watermark --iversions

sys   : 3.7.11 (default, Jul 27 2021, 07:03:16) 
[Clang 10.0.0 ]
pandas: 1.3.4
numpy : 1.21.2
re    : 2.2.1

